在深度学习中，原数据集往往很大，不可能一次性的全部载入模型，训练模型都是小批量小批量地优化训练的，那么 PyTorch 是如何批量地加载数据的呢？

PyTorch 的 torch.utils.data 包中定义了两个类 Dataset 和 DataLoader，通过学习这两个类，我们就会明白 PyTorch 是如何批量地加载数据的。

<br/>

# 1. 分割数据集
 
在使用 Dataset 和 DataLoader 处理数据之前，需要将数据划分为训练集、验证集和测试集。

详细过程请参考 `./1_split.py`

<br/>

# 2. Dataset（我的理解：Dataset 子类用于将数据整理为一定格式的数据）

Pytorch 中给定的 Dataset 类是一个抽象类，只能用于继承，我们自定义的 Dataset 子类需要继承它，并且重写 `__getitem__()`；

在自定义的 Dataset 子类中，`__getitem__()` 接收索引，返回数据，接收的索引 index 由 DataLoader() 提供，返回的数据由 Dataset 子类中的定义的方法整理好。

<br/>

**2.1 下面根据人民币二分类任务中自定义的 RMBDataset 子类的代码为例介绍自定义 Dataset 子类的框架：**

完整代码请查看 ../tools/my_dataset.py 中的 RMBDataset()

```
from torch.utils.data import Dataset

class RMBDataset(Dataset):
    # 初始化
    def __init__(self, data_dir):
        self.label_name = {"1": 0, "100": 1}
        self.data_info = self.get_img_info(data_dir)

    # 定义 __getitem__()，接收索引，根据索引获取 data_info 中的数据，返回图片和 label 标签
    def __getitem__(self, index):
        path_img, label = self.data_info[index]
        img = Image.open(path_img).convert('RGB')
        return img, label

    def __len__(self):
        return len(self.data_info)

    # 定义 get_img_info()，输入数据路径 data_dir，返回 data_info，data_info 是一个由元组组成的列表，每个元组包含图片路径和 label 标签两个元素
    @staticmethod
    def get_img_info(data_dir):
        data_info = list()
        path_img = ···
        label = ···
        data_info.append((path_img, int(label)))
        return data_info
```

<br/>

**2.2 Dataset 子类的数据逻辑：**

data_dir ——> get_img_info(data_dir) ——> return data_info ——> `__getitem__(index)` ——> return img, label

<br/>

# 3. DataLoader（我的理解：DataLoader 用于按照一定的规则加载数据）

DataLoader() 用于构建可迭代的数据装载器，使用时需要实例化。

**3.1 四个主要参数：**

batch_size：批大小，默认值为 1

num_works：读取数据的进程数，默认值为 0，0 means that the data will be loaded in the main process. 

shuffle：每个 epoch 是否打乱顺序，默认值为 Flase

drop_last：当样本数不能被 batchsize 整除时，是否舍弃最后一批数据，默认值为 Flase

<br/>

**3.2 Epoch Iteration Batchsize 之间的关系：**

Epoch：一个 Epoch 指的是整个训练数据集被完整地用来训练一次；

Batchsize：是指每次传递给模型进行训练的数据样本数；

Iteration：是指在训练过程中模型权重更新的总次数；

例如：如果一个数据集中有 2000 张图片，设置 Batchsize 为 500，则一个 Epoch 中的 Iteration 数为 2000/500 = 4，这意味着跑完一个 Epoch 需要迭代 4 个 Batch。

<br/>

**3.3 下面根据训练模型的数据准备部分的代码为例介绍 RMBDataset() 实例化和使用 DataLoader() 的框架：**

完整代码请查看 ./2_train_lenet.py 中的数据准备部分

```
from torch.utils.data import DataLoader

train_dir = ···

# 构建 RMBDataset() 实例
train_data = RMBDataset(data_dir=train_dir)

# DataLoder() 实例化，构建数据加载器
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
```

**1) RMBDataset 子类实例化的数据逻辑**

`train_data = RMBDataset(data_dir=train_dir)`

train_data：RMBDataset() 的 `__getitem__(index)` 根据 index 读取 data_info 数据后返回图片和 label 标签，然后赋值给 train_data；

**2) `__getitem__(index)` 的 index 哪里来呢？**

`train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)`

DataLoader() 会根据 batch_size 和 num_works 的值调用采样器 samper() 返回 index 数据。

<br/>

**3.4 DataLoader 的数据逻辑：**

根据 batch_size 和 num_works 的值 ——> index ——> `__getitem__(index)` ——> return img, label ——> train_data ——> train_loader（多 batch 数据）

train_loader 包含多 batch 数据，多 batch 数据存放在一个列表中，每一批数据都是一个包含两个元素的列表，一个元素是图片，另外一个元素是 label 标签，图片和标签也是以列表的形式存放的，所以数据形状为`[[[图片1], [标签1]], [[图片2], [标签2]], [[图片3], [标签3]]···]`。

<br/>

**3.5 我的理解：**
Dataset 子类用于将数据整理为一定格式的数据，DataLoader 用于按照一定的规则加载数据。

<br/>

# 5. 迭代训练模型

**框架：**
```
for epoch in range(MAX_EPOCH):

    for i, data in enumerate(train_loader):  # 循环地从 train_loader 中获取一个 batch_size 大小的数据

        # forward
        inputs, labels = data
        outputs = net(inputs)

        # backward
        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()

        # update weights
        optimizer.step()
```